##  Imports

In [1]:
from torchvision import models,transforms,datasets
import torch
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os
import torch.nn as nn
from torch.autograd import Variable
import torch
import bcolz
import time
from utils import *
#import tensorflow as tf
#import keras
%matplotlib inline

In [15]:
torch.__version__

'0.1.11+8aa1cef'

## Data processing

In [10]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

prep1 = transforms.Compose([
            transforms.RandomSizedCrop(224),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            normalize,
        ])

In [11]:
data_dir = 'dogsandcats/'

dsets = {x: datasets.ImageFolder(os.path.join(data_dir, x), prep1)
         for x in ['train', 'val']}

dset_loaders = {x: torch.utils.data.DataLoader(dsets[x], batch_size=64,
                                               shuffle=False, num_workers=6)
                for x in ['train', 'val']}
dset_sizes = {x: len(dsets[x]) for x in ['train', 'val']}
dset_classes = dsets['train'].classes

use_gpu = torch.cuda.is_available()

## Creating VGG Model

In [4]:
model_vgg = models.vgg16(pretrained=True)


### Modifying last layer and setting the gradient false to all layers

In [5]:
for param in model_vgg.parameters():
    param.requires_grad = False
model_vgg.classifier[6].out_features = 2
for param in model_vgg.classifier[6].parameters():
    param.requires_grad = True

In [6]:
model_vgg = model_vgg.cuda()

### Calculating preconvoluted features

In [7]:
def preconvfeat(dataset):
    conv_features = []
    labels_list = []
    for data in dataset:
        inputs,labels = data

        inputs , labels = Variable(inputs.cuda()),Variable(labels.cuda())
        x = model_vgg.features(inputs)
        conv_features.extend(x.data.cpu().numpy())
        labels_list.extend(labels.data.cpu().numpy())
    conv_features = np.concatenate([[feat] for feat in conv_features])
    return (conv_features,labels_list)

In [8]:
%%time
conv_feat_train,labels_train = preconvfeat(dset_loaders['train'])

CPU times: user 53.2 s, sys: 14.4 s, total: 1min 7s
Wall time: 1min 8s


In [9]:
%%time
conv_feat_val,labels_val = preconvfeat(dset_loaders['val'])

CPU times: user 4.8 s, sys: 1.33 s, total: 6.13 s
Wall time: 6.56 s


In [10]:
save_array('conv_feat_train.bc',conv_feat_train)
save_array('labels_train.bc',labels_train)
save_array('conv_feat_val.bc',conv_feat_val)
save_array('labels_val.bc',labels_val)


### Loading Preconvoluted features

In [6]:
conv_feat_train = load_array('conv_feat_train.bc')
labels_train = load_array('labels_train.bc')
conv_feat_val = load_array('conv_feat_val.bc')
labels_val = load_array('labels_val.bc')

In [8]:
conv_feat_train.shape

(23000, 512, 7, 7)

## Training fully connected module

### Creating loss function and optimizer

In [11]:
criterion = nn.CrossEntropyLoss()
lr = 0.01
optimizer = torch.optim.SGD(model_vgg.classifier[6].parameters(),lr = lr)

### Creating Data generator

In [12]:
def data_gen(conv_feat,labels,batch_size=64,shuffle=True):
    labels = np.array(labels)
    if shuffle:
        index = np.random.permutation(len(conv_feat))
        conv_feat = conv_feat[index]
        labels = labels[index]
    for idx in range(0,len(conv_feat),batch_size):
        yield(conv_feat[idx:idx+batch_size],labels[idx:idx+batch_size])

### Training the model

In [13]:
def train_model(model,size,conv_feat=None,labels=None,epochs=1,optimizer=None,train=True,shuffle=True):
    for epoch in range(epochs):
        batches = data_gen(conv_feat=conv_feat,labels=labels,shuffle=shuffle)
        total = 0
        running_loss = 0.0
        running_corrects = 0
        for inputs,classes in batches:
            inputs , classes = Variable(torch.from_numpy(inputs).cuda()),Variable(torch.from_numpy(classes).cuda())
            inputs = inputs.view(inputs.size(0), -1)
            outputs = model(inputs)
            loss = criterion(outputs,classes)           
            if train:
                if optimizer is None:
                    raise ValueError('Pass optimizer for train mode')
                optimizer = optimizer
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
            _,preds = torch.max(outputs.data,1)
            # statistics
            running_loss += loss.data[0]
            running_corrects += torch.sum(preds == classes.data)
        epoch_loss = running_loss / size
        epoch_acc = running_corrects / size
        print('Loss: {:.4f} Acc: {:.4f}'.format(
                     epoch_loss, epoch_acc))
    

In [15]:
%%time
(train_model(model=model_vgg.classifier,size=dset_sizes['train'],conv_feat=conv_feat_train,labels=labels_train,
            epochs=10,optimizer=optimizer,train=True,shuffle=True))

Loss: 0.0016 Acc: 0.9570
Loss: 0.0016 Acc: 0.9590
Loss: 0.0015 Acc: 0.9597
Loss: 0.0015 Acc: 0.9617
Loss: 0.0015 Acc: 0.9598
Loss: 0.0015 Acc: 0.9607
Loss: 0.0014 Acc: 0.9621
Loss: 0.0015 Acc: 0.9627
Loss: 0.0014 Acc: 0.9619
Loss: 0.0014 Acc: 0.9610
CPU times: user 20.4 s, sys: 5.04 s, total: 25.5 s
Wall time: 25.4 s


In [34]:
train_model(conv_feat=conv_feat_val,labels=labels_val,model=model_vgg.classifier
            ,size=dset_sizes['val'],train=False,shuffle=True)

Loss: 0.0015 Acc: 0.9665


### Training all the last Linear layers.

In [19]:
for param in model_vgg.classifier.parameters():
    param.requires_grad = True

In [20]:
criterion = nn.CrossEntropyLoss()
lr = 0.01
optimizer = torch.optim.SGD(model_vgg.classifier.parameters(),lr = lr)

In [21]:
train_model(conv_feat=conv_feat_train,labels=labels_train,model=model_vgg.classifier
            ,size=dset_sizes['train'],epochs=5,optimizer=optimizer,train=True,shuffle=True)

Loss: 0.0014 Acc: 0.9608
Loss: 0.0012 Acc: 0.9678
Loss: 0.0010 Acc: 0.9722
Loss: 0.0009 Acc: 0.9764
Loss: 0.0008 Acc: 0.9797


In [22]:
def update_lr_by(value=0.1,optimizer=None):
    if optimizer is None:
        raise ValueError('Pass in a valid optimizer')
    optimizer = optimizer
    for param_group in optimizer.param_groups:
        param_group['lr'] = param_group['lr'] * 0.1
        print('updated learning rate to {}'.format(param_group['lr']))
    return optimizer

In [23]:
optimizer = update_lr_by(optimizer=optimizer)

updated learning rate to 0.001


In [24]:
train_model(conv_feat=conv_feat_train,labels=labels_train,model=model_vgg.classifier
            ,size=dset_sizes['train'],epochs=5,optimizer=optimizer,train=True,shuffle=True)

Loss: 0.0006 Acc: 0.9836
Loss: 0.0007 Acc: 0.9833
Loss: 0.0006 Acc: 0.9853


### Changing dropout

In [25]:
model_vgg.classifier[5].p = 0.2
model_vgg.classifier[2].p = 0.2

In [26]:
train_model(conv_feat=conv_feat_train,labels=labels_train,model=model_vgg.classifier
            ,size=dset_sizes['train'],epochs=5,optimizer=optimizer,train=True,shuffle=True)

Loss: 0.0004 Acc: 0.9920
Loss: 0.0004 Acc: 0.9922
Loss: 0.0004 Acc: 0.9936


### Add Batch Nomralization

In [9]:
models_vgg_bn = models.vgg16_bn()
models_vgg_bn = models_vgg_bn.cuda()

In [10]:
criterion = nn.CrossEntropyLoss()
lr = 0.001
optimizer = torch.optim.SGD(models_vgg_bn.classifier.parameters(),lr = lr)

In [14]:
train_model(conv_feat=conv_feat_train,labels=labels_train,model=models_vgg_bn.classifier
            ,size=dset_sizes['train'],epochs=50,optimizer=optimizer,train=True,shuffle=True)

Loss: 0.0078 Acc: 0.8325
Loss: 0.0033 Acc: 0.9153
Loss: 0.0026 Acc: 0.9296
Loss: 0.0024 Acc: 0.9351
Loss: 0.0022 Acc: 0.9416
Loss: 0.0021 Acc: 0.9430
Loss: 0.0020 Acc: 0.9470
Loss: 0.0019 Acc: 0.9504
Loss: 0.0018 Acc: 0.9514
Loss: 0.0018 Acc: 0.9523
Loss: 0.0017 Acc: 0.9540
Loss: 0.0017 Acc: 0.9563
Loss: 0.0016 Acc: 0.9580
Loss: 0.0015 Acc: 0.9595
Loss: 0.0015 Acc: 0.9611
Loss: 0.0015 Acc: 0.9607
Loss: 0.0014 Acc: 0.9632
Loss: 0.0014 Acc: 0.9634
Loss: 0.0013 Acc: 0.9650
Loss: 0.0013 Acc: 0.9647
Loss: 0.0013 Acc: 0.9673
Loss: 0.0013 Acc: 0.9677
Loss: 0.0012 Acc: 0.9685
Loss: 0.0012 Acc: 0.9675
Loss: 0.0012 Acc: 0.9703
Loss: 0.0011 Acc: 0.9704
Loss: 0.0011 Acc: 0.9707
Loss: 0.0011 Acc: 0.9717
Loss: 0.0011 Acc: 0.9729
Loss: 0.0010 Acc: 0.9731
Loss: 0.0010 Acc: 0.9749
Loss: 0.0010 Acc: 0.9733
Loss: 0.0010 Acc: 0.9753
Loss: 0.0009 Acc: 0.9776
Loss: 0.0009 Acc: 0.9762
Loss: 0.0009 Acc: 0.9783
Loss: 0.0009 Acc: 0.9766
Loss: 0.0009 Acc: 0.9776
Loss: 0.0008 Acc: 0.9798
Loss: 0.0008 Acc: 0.9799


In [ ]:
train_model(conv_feat=conv_feat_val,labels=labels_val,model=models_vgg_bn.classifier
            ,size=dset_sizes['val'],train=False,shuffle=False)

### Adjusting dropout

In [58]:
models_vgg_bn.classifier[5].p = 0.3
models_vgg_bn.classifier[2].p = 0.3

In [53]:
optimizer = update_lr_by(value=0.1,optimizer=optimizer)

updated learning rate to 1e-05


In [61]:
train_model(conv_feat=conv_feat_train,labels=labels_train,model=models_vgg_bn.classifier
            ,size=dset_sizes['train'],epochs=50,optimizer=optimizer,train=True,shuffle=True)

Loss: 0.0014 Acc: 0.9638
Loss: 0.0014 Acc: 0.9647
Loss: 0.0014 Acc: 0.9641
Loss: 0.0014 Acc: 0.9652
Loss: 0.0014 Acc: 0.9647
Loss: 0.0014 Acc: 0.9663
Loss: 0.0014 Acc: 0.9651
Loss: 0.0014 Acc: 0.9660
Loss: 0.0014 Acc: 0.9653
Loss: 0.0014 Acc: 0.9644


In [62]:
train_model(conv_feat_val,labels_val,models_vgg_bn.classifier,dset_sizes['val'],train=False,shuffle=True)

Loss: 0.0016 Acc: 0.9590


## Training a model without preconvolution layer

In [4]:
model_vgg = models.vgg16(pretrained=True)
model_vgg = model_vgg.cuda()

In [5]:
for param in model_vgg.features.parameters():
    param.requires_grad = False

In [6]:
model_vgg.classifier[6].out_features = 2

In [7]:
criterion = nn.CrossEntropyLoss()
lr = 0.01
optimizer = torch.optim.SGD(model_vgg.classifier.parameters(),lr=lr)

In [8]:
%%time
train=True
for epoch in range(10):
        #batches = data_gen(conv_feat=conv_feat,labels=labels,shuffle=shuffle)
        total = 0
        running_loss = 0.0
        running_corrects = 0
        for inputs,classes in dset_loaders['train']:
            inputs , classes = Variable(inputs.cuda()),Variable(classes.cuda())
            #inputs = inputs.view(inputs.size(0), -1)
            outputs = model_vgg(inputs)
            loss = criterion(outputs,classes)           
            if train:
                if optimizer is None:
                    raise ValueError('Pass optimizer for train mode')
                optimizer = optimizer
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
            _,preds = torch.max(outputs.data,1)
            # statistics
            running_loss += loss.data[0]
            running_corrects += torch.sum(preds == classes.data)
        epoch_loss = running_loss / dset_sizes['train']
        epoch_acc = running_corrects / dset_sizes['train']
        print('Loss: {:.4f} Acc: {:.4f}'.format(
                     epoch_loss, epoch_acc))

Loss: 0.0035 Acc: 0.9739
Loss: 0.0005 Acc: 0.9927
Loss: 0.0004 Acc: 0.9933
Loss: 0.0003 Acc: 0.9931
Loss: 0.0004 Acc: 0.9936
Loss: 0.0003 Acc: 0.9940
Loss: 0.0003 Acc: 0.9938
Loss: 0.0003 Acc: 0.9933
Loss: 0.0003 Acc: 0.9936
Loss: 0.0003 Acc: 0.9938
CPU times: user 10min 6s, sys: 2min 10s, total: 12min 17s
Wall time: 15min 59s


### Using single thread

In [12]:
%%time
train=True
for epoch in range(10):
        #batches = data_gen(conv_feat=conv_feat,labels=labels,shuffle=shuffle)
        total = 0
        running_loss = 0.0
        running_corrects = 0
        for inputs,classes in dset_loaders['train']:
            inputs , classes = Variable(inputs.cuda()),Variable(classes.cuda())
            #inputs = inputs.view(inputs.size(0), -1)
            outputs = model_vgg(inputs)
            loss = criterion(outputs,classes)           
            if train:
                if optimizer is None:
                    raise ValueError('Pass optimizer for train mode')
                optimizer = optimizer
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
            _,preds = torch.max(outputs.data,1)
            # statistics
            running_loss += loss.data[0]
            running_corrects += torch.sum(preds == classes.data)
        epoch_loss = running_loss / dset_sizes['train']
        epoch_acc = running_corrects / dset_sizes['train']
        print('Loss: {:.4f} Acc: {:.4f}'.format(
                     epoch_loss, epoch_acc))

Loss: 0.0003 Acc: 0.9935
Loss: 0.0003 Acc: 0.9936
Loss: 0.0003 Acc: 0.9940
Loss: 0.0003 Acc: 0.9940
Loss: 0.0003 Acc: 0.9934
Loss: 0.0003 Acc: 0.9934
Loss: 0.0003 Acc: 0.9932
Loss: 0.0003 Acc: 0.9937
Loss: 0.0003 Acc: 0.9933
Loss: 0.0003 Acc: 0.9934
CPU times: user 9min 32s, sys: 2min 11s, total: 11min 44s
Wall time: 11min 48s
